---
# IMPORT

In [46]:
import os
import numpy as np
import pandas as pd
import ast

from PIL import Image

from tensorflow.keras.models import Model
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dropout, Dense, Input, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam

---
# VARIABLES

In [47]:
mapping = {0:'masked', 1:'unmasked'}
model_name = 'Masked_Model_Best.h5'#path model 
bbox_label_file = 'path_csv.csv'#path csv
link_folder = 'data'#path folder
size_image = (128, 128)


---
# LOAD MODEL

In [48]:
def build_model():
    inputs = Input(shape=(128,128,3))

    base_model = EfficientNetB0(include_top=False, weights='imagenet',
                                drop_connect_rate=0.33, input_tensor=inputs)
    x = GlobalAveragePooling2D()(base_model.output)
    x = Dropout(.5, name="top_dropout")(x)
    outputs = Dense(2, activation='softmax')(x)

    model = Model(inputs, outputs)

    model.compile(optimizer=Adam(learning_rate=1e-3),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [49]:
model = build_model()
model.load_weights(model_name)
model

---
# PREDICT TEST

In [50]:
def read_file_image(file_path):
    test_data = []
    label_file = pd.read_csv(bbox_label_file)
    for i in range(len(label_file)):
        pixels = Image.open(os.path.join(file_path, label_file.iloc[i]['file_name']))
        bbox = label_file.iloc[i]['bbox']
        bbox = ast.literal_eval(bbox)
        pixels = pixels.crop((bbox[0], bbox[1], bbox[0] + bbox[2], bbox[1] + bbox[3]))
        test_data.append(np.array(pixels.resize(size_image)))
    return test_data

In [51]:
def preprocess_data_input(data):
    processed_data = []
    for each in data:
        if len(each.shape) == 2:
            each = np.expand_dims(each, axis=-1)
            each = np.concatenate([each, each, each], axis=-1)
        elif each.shape[2] == 1:
            each = np.concatenate([each, each, each], axis=-1)
        processed_data.append(each)
    return np.array(processed_data)

In [52]:
# Read data
data = read_file_image(link_folder)
processed_data = preprocess_data_input(data)

In [53]:
labels = model.predict(processed_data)
labels = np.argmax(np.array(labels), axis=1)

877/877 [==============================] - 313s 354ms/step


In [54]:
bbox_labels = pd.read_csv('path_csv.csv')#path csv
result = pd.DataFrame({'file_name': bbox_labels['file_name'], 'bbox': bbox_labels['bbox'], 'masked': list(map(lambda x: mapping[x], labels))})
result.to_csv('masked_result', index=False) #result file csv